In [7]:
import numpy as np
import tensorflow as tf

#### Fetch the openml digit recognition data

In [8]:
from sklearn.datasets import fetch_openml
data = fetch_openml('mnist_784', as_frame=False)
X, y = data.data, data.target

In [9]:
X.shape

(70000, 784)

In [10]:
y.shape

(70000,)

#### Splitting the data into x_train, y_train, x_test, y_test:

In [12]:
x_train = X[:60000]
y_train = y[:60000]

x_test = X[60000:]
y_test = y[60000:]

##### Converting the datset x_train and x_test dataset into 3 channels

In [16]:
x_train = x_train.reshape(-1, 28, 28)
x_test = x_test.reshape(-1, 28, 28)

x_train = x_train[:, :, :, np.newaxis]
x_test  = x_test[:, :, :, np.newaxis]

x_train = np.repeat(x_train, 3, axis=3)
x_test = np.repeat(x_test, 3, axis=3)

##### Adding zero padding to the training dataset

In [17]:
# Creating the function to add zero padding to the training dataset

def zero_pad(X, pad):
    # Adding the zero padding of size pad to the height and width of X

    X_pad = np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)), mode = 'constant', constant_values = (0, 0))
    # returning X_pad
    return X_pad

#### Single step convolution

Here we are creating single step convolution operation where kernel window is applied on the previous input image and multiply and added the all the pixel values from the input image. It will result into the output which is reduced version of the input image.

In [18]:
def conv_single_step(a_prev_slice, W, b):
    #apply window W over a_prev_slice
    s = a_prev_slice * W
    Z = np.sum(s) + float(b)
    Z = np.float64(Z)

    return Z

#### Forward propagation in convolution neural network

In forward convolution forward prop we building the function to convolve the kernal over the previous layer activation. Also defining the output shape of convolution to store the Conv_single _step into the output.

In [19]:
def conv_forward(A_prev, W, b, hparameters):

    # Retrieve the dimensions of the A_prev's shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve the dimensions of the W shape
    (f, f, n_C_prev, n_C) = W.shape

    #Retrieve strides and pad from hparameters
    strides = hparameters["stride"]
    pad = hparameters["pad"]

    #Declare the dimensions of the output and Create the output with the given dimension
    n_H = int((n_H_prev - f + 2 * pad)/strides) + 1
    n_W = int((n_W_prev - f + 2 * pad)/strides) + 1

    Z = np.zeros((m, n_H, n_W, n_C))

    #Add zero padding to the previous layer activation using zero_pad function
    A_prev_pad = zero_pad(A_prev, pad)

    #Loop over the batch of training examples
    for i in range(m):
        a_prev_pad = A_prev_pad[i]   #selecting the single example

        #Loop over the vertical axis of output volume
        for h in range(n_H):
            vert_start = strides * h   #defining the vertical start and end
            vert_end = vert_start + f

            #Loop over the horizontal axis of output volumn
            for w in range(n_W):
                horiz_start = strides * w
                horiz_end = horiz_start + f

                #Loop over the number of filters
                for c in range(n_C):
                    #selecting the slice from the a_prev_pad to convolve the window
                    a_prev_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    #selecting the parameters W and b and perform Conv_single_step
                    weights = W[:, :, :, c]
                    biases = b[:, :, :, c]
                    Z[i, h, w, c] = conv_single_step(a_prev_slice, weights, biases)
    
    #Save information in cache for backprop 
    cache = (A_prev, W, b, hparameters)
    return Z, cache

### Forward pooling in convolutional neural network

The pooling layer reduces the height and width of the input. It helps to reduce the computatuion.

In [21]:
#Implementing forward pooling function
def pool_forward(A_prev, hparameters, mode = "max"):

    #Retrieve the dimensions of the previous activation
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    #Retrieve hyperparameters from hparameters
    f = hparameters["f"]
    stride = hparameters["stride"]

    #Define Dimensions of the output
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev

    #Initialize the output matrix A
    A = np.zeros((m, n_H, n_W, n_C))

    #Loop over the input examples in A_prev
    for i in range(m):
        a_prev = A_prev[i]

        #Loop over the vertical axis of output volumn
        for h in range(n_H):
            vert_start = stride * h
            vert_end = vert_start + f

            #Loop over the horizontal axis of output volumn
            for w in range(n_W):
                horiz_start = stride * w
                horiz_end = horiz_start + f

                #Loop over the number of channels of output volumn
                for c in range(n_C):
                     a_prev_slice = a_prev[
                        vert_start:vert_end, 
                        horiz_start:horiz_end, 
                        c
                        ]

                     if mode == "max":
                         A[i, h, w, c] = np.max(a_prev_slice)
                     elif mode == "average":
                         A[i, h, w, c] = np.mean(a_prev_slice)

    cache = (A_prev, hparameters)

    return A, cache            